In [3]:
import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [4]:
data =  datasets.load_dataset('arcd')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/693 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/702 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 693
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 702
    })
})

In [ ]:
train_examples = data['train']

In [ ]:
train_examples

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 693
})

In [ ]:
df = pd.DataFrame(data['train'])

In [ ]:
df.head(10)

,id,title,context,question,answers
0,969331847966,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- من هو جمال أحمد حمزة خاشقجي؟,"{'text': ['صحفي وإعلامي'], 'answer_start': [73]}"
1,115150665555,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال,{'text': ['حمزة خاشقجي (13 أكتوبر 1958، المدين...
2,74212080718,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال,"{'text': ['المدينة المنورة'], 'answer_start': ..."
3,465699296586,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال,"{'text': ['واشنطن بوست'], 'answer_start': [224]}"
4,564177542570,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال,{'text': ['وُصف في الصحف وأجهزة الاعلام العالم...
5,98945395462,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي منصب شغل في الجريدة؟ ال,"{'text': ['وتقلّد منصب مستشار،'], 'answer_star..."
6,838329318053,جمال خاشقجي,غادر خاشقجي السعودية في سبتمبر 2017، وكتب بعد ...,- متى غادر خاشقجي السعودية؟ ال,"{'text': ['في سبتمبر 2017،'], 'answer_start': ..."
7,432910739583,جمال خاشقجي,غادر خاشقجي السعودية في سبتمبر 2017، وكتب بعد ...,- لمن ينتقده في مقالاته الإخبارية؟ ال,"{'text': ['الحكومة السعودية.'], 'answer_start'..."
8,619452973730,جمال خاشقجي,غادر خاشقجي السعودية في سبتمبر 2017، وكتب بعد ...,- لمن انتقد خاشقجي بشدة في مقالاته الإخبارية؟ ال,{'text': ['ولي العهد السعودي محمد بن سلمان، وا...
9,333772766499,السعودية,السعودية أو (رسميًا: المملكة العربية السعودية)...,- أي دولة هي أكبر دولة في الشرق الأوسط؟ ال,"{'text': ['المملكة العربية السعودية)'], 'answe..."


In [ ]:
df.question[250]

'ماهي المهنة التي كان يمارسها الفينيقيون؟'

In [ ]:
df.shape

(693, 5)

#### Prétraitement:

In [ ]:
#Enlever la ponctuation
import string

def delete_punct(text):
    return text.translate(str.maketrans('','',string.punctuation))

In [ ]:
#Stopwords Removal
import nltk
from nltk.corpus import stopwords

arabic_stopwords = stopwords.words('arabic')

def delete_stopwords(text):
    for w in text:
        if w in arabic_stopwords:
            text.replace(w,'')
    return text

In [ ]:
df['question']=[delete_punct(q) for q in df.question]
df.question = [delete_stopwords(q) for q in df.question]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(df['question'])
#tokenizer.get_config()

vocab_size = len(tokenizer.word_index)
print('number of unique words = ', vocab_size)

number of unique words =  1698


In [ ]:
ptrn2seq = tokenizer.texts_to_sequences(df['question']) #convertir le texte en séquences d'entiers
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['context'])

print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

X shape =  (693, 15)
y shape =  (693,)
num of classes =  231


La première couche Dense avec une fonction d'activation ReLU permet d'appliquer une transformation linéaire aux sorties de la dernière couche LSTM, afin de produire une représentation plus dense et plus informative de la séquence de texte. Cela permet de capturer des relations non-linéaires entre les entrées et les sorties, ce qui peut améliorer la capacité de classification du modèle.

La deuxième couche Dense avec une fonction d'activation ReLU permet de continuer à capturer des relations non-linéaires entre les entrées et les sorties, en appliquant une transformation linéaire à la sortie de la première couche Dense. Cela peut aider à augmenter la complexité du modèle et à capturer des relations plus complexes entre les entrées et les sorties.

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
#from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(Input(shape=(X.shape[1])))
model.add(Embedding(input_dim=vocab_size+1, output_dim=100, mask_zero=True))
model.add(LSTM(64, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(64,return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(64))
model.add(LayerNormalization())
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation="softmax"))
#la couche de sortie avec une fonction d'activation softmax permet de produire une distribution
#de probabilité sur les différentes classes de sortie.
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
#l'optimiseur 'adam' est une méthode d'optimisation de gradient stochastique

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 100)           169900    
                                                                 
 lstm (LSTM)                 (None, 15, 64)            42240     
                                                                 
 layer_normalization (LayerN  (None, 15, 64)           128       
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 layer_normalization_1 (Laye  (None, 15, 64)           128       
 rNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 64)                3

In [ ]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=10,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=100)

Epoch 1/100
70/70 [==============================] - 20s 31ms/step - loss: 5.8657 - accuracy: 0.0000e+00
Epoch 2/100
70/70 [==============================] - 2s 30ms/step - loss: 5.2762 - accuracy: 0.0188
Epoch 3/100
70/70 [==============================] - 2s 30ms/step - loss: 4.5748 - accuracy: 0.0938
Epoch 4/100
70/70 [==============================] - 2s 30ms/step - loss: 3.9275 - accuracy: 0.2179
Epoch 5/100
70/70 [==============================] - 2s 30ms/step - loss: 3.2753 - accuracy: 0.3636
Epoch 6/100
70/70 [==============================] - 2s 31ms/step - loss: 2.6151 - accuracy: 0.5455
Epoch 7/100
70/70 [==============================] - 2s 29ms/step - loss: 2.0114 - accuracy: 0.6696
Epoch 8/100
70/70 [==============================] - 2s 30ms/step - loss: 1.5549 - accuracy: 0.7677
Epoch 9/100
70/70 [==============================] - 2s 29ms/step - loss: 1.1162 - accuracy: 0.8586
Epoch 10/100
70/70 [==============================] - 2s 29ms/step - loss: 0.7923 - accuracy: 0

In [ ]:
import openai
#put your openai key below
openai.api_key = ""

def generate_response(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    return response.choices[0].text.strip()

In [ ]:
import re
import random

def model_responce(query):

    text = []
    txt = delete_punct(query)
    txt = delete_stopwords(txt)
    txt = query.split()
    txt = " ".join(txt)
    text.append(txt)
    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    print(tag)
    responses = df[df['context'] == tag]['answers']

    try:
        response = str(random.choice(list(responses))['text'])
        return (response)

    except Exception:
        print("you: {}".format(query))
        print("model: {}".format(generate_response(query)))
        return generate_response(query)


In [ ]:

print(model_responce('  متى ولد جمال أحمد حمزة خاشقجي وتوفي؟'))


1/1 [==============================] - 5s 5s/step
جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّة مناصب لعدد من الصحف في السعودية، وتقلّد منصب مستشار، كما أنّه مدير عام قناة العرب الإخبارية سابقًا.
['حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)،']


In [ ]:
print(model_responce( 'الحدود في الشرق'))

1/1 [==============================] - 0s 26ms/step
السعودية ورسميًا المملكة العربية السعودية هي أكبر دولة في الشرق الأوسط وتقع تحديدًا في الجنوب الغربي من قارة آسيا وتشكل الجزء الأكبر من شبه الجزيرة العربية إذ تبلغ مساحتها حوالي مليوني كيلومتر مربع. يحدها من الشمال العراق والأردن وتحدها الكويت من الشمال الشرقي، ومن الشرق تحدها كل من قطر والإمارات العربية المتحدة بالإضافة إلى البحرين التي ترتبط بالسعودية من خلال جسر الملك فهد الواقع على الخليج العربي، ومن الجنوب تحدها اليمن، وعُمان من الجنوب الشرقي، كما يحدها البحر الأحمر من جهة الغرب.
['قطر والإمارات العربية المتحدة بالإضافة إلى البحرين التي ترتبط بالسعودية من خلال جسر الملك']


In [ ]:
print(model_responce('hello'))


1/1 [==============================] - 0s 26ms/step
رجب طيب أردوغان (بالتركية: Recep Tayyip Erdoğan)‏ (ولد في 26 فبراير 1954)، هو رئيس جمهورية تركيا الثاني عشر والحالي منذ 28 أغسطس 2014م، ويعد أول رئيس تركي اختاره الشعب بطريق الاقتراع المباشر، انتخب رئيساً للجمهورية مرة ثانية في انتخابات مبكرة في 24 يونيو 2018 وتسلم سلطاته في 9 يوليو 2018 ليصبح أول رئيس تركي يدشن النظام الرئاسي .
['اختاره الشعب بطريق الاقتراع المباشر،']


In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    print(type(model_responce(userText)))
    return model_responce(userText)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
